In [1]:
import sys
from elasticsearch.helpers import BulkIndexError
import json
from elasticsearch import helpers,Elasticsearch
from dotenv import load_dotenv
import os
import pandas as pd

In [7]:
load_dotenv()


True

In [2]:



client = Elasticsearch(
    ["https://localhost:9200"],  # Địa chỉ Elasticsearch
    basic_auth=("elastic", os.getenv('ELASTIC_PASSWORD')),  # Thay your_password bằng ELASTIC_PASSWORD
    # verify_certs=False  # Bỏ kiểm tra SSL (hoặc cung cấp đường dẫn CA nếu cần)
    ca_certs="ca.crt",
    request_timeout=60

)


In [3]:
client.info()

ConnectionError: Connection error caused by: ConnectionError(Connection error caused by: NewConnectionError(<elastic_transport._node._urllib3_chain_certs.HTTPSConnection object at 0x000001F2A33B7FB0>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it))

In [4]:
client.ping()

True

In [5]:
labels = pd.read_csv("data/labels.csv")
labels

,scenario,attack,start,end
0,russellmitchell,network_scans,1642993260,1642996606
1,russellmitchell,service_scans,1642996606,1642996645
2,russellmitchell,dirb,1642996645,1642996668
3,russellmitchell,wpscan,1642996668,1642996699
4,russellmitchell,webshell,1642996699,1642996762
...,...,...,...,...
74,wilson,cracking,1644232839,1644234460
75,wilson,reverse_shell,1644234460,1644234498
76,wilson,privilege_escalation,1644234498,1644234533
77,wilson,service_stop,1644144435,1644144437


In [7]:
print(labels['scenario'].unique())

['russellmitchell' 'fox' 'harrison' 'santos' 'shaw' 'wardbeck' 'wheeler'
 'wilson']


In [15]:

def get_all_file_names(folder_path):
    try:
        # List all files in the folder and remove ".json" extension
        file_names = [
            os.path.splitext(file)[0] for file in os.listdir(folder_path)
            if os.path.isfile(os.path.join(folder_path, file)) and (file.endswith(".json") or file.endswith(".csv"))
        ]
        return file_names
    except FileNotFoundError:
        print(f"The folder '{folder_path}' does not exist.")
        return []

In [14]:
folder_path = "data/AIT_alert"
output_path = "data/AIT_labeled"
file_names = get_all_file_names(folder_path)
print("Files in folder:", file_names)

Files in folder: ['fox_aminer', 'fox_wazuh', 'harrison_aminer', 'harrison_wazuh', 'russellmitchell_aminer', 'russellmitchell_wazuh', 'santos_aminer', 'santos_wazuh', 'shaw_aminer', 'shaw_wazuh', 'wardbeck_aminer', 'wardbeck_wazuh', 'wheeler_aminer', 'wheeler_wazuh', 'wilson_aminer', 'wilson_wazuh']


In [ ]:
from datetime import datetime

for scenario in labels['scenario'].unique():
    for file in file_names:
        if file.startswith(scenario):
            labels_system = labels[labels['scenario'] == scenario]
            with open(os.path.join(folder_path, f"{file}.json"), 'r') as f:
                # Đọc từng dòng trong file
                for line in f:
                    # Chuyển đổi dòng thành JSON
                    data_json = json.loads(line.strip())
                    # Lấy timestamp của document
                    if file.__contains__("aminer"):
                        timestamp = data_json["LogData"]["Timestamps"][0]
                    else :
                        # continue
                        iso_timestamp = data_json["@timestamp"]  # VD: "2022-01-15T02:32:32.000000Z"
                        dt = datetime.strptime(iso_timestamp, "%Y-%m-%dT%H:%M:%S.%fZ")
                        timestamp = dt.timestamp()

                    # Gán nhãn phù hợp hoặc "unknown" nếu không khớp
                    label_assigned = "benign"
                    for index, row in labels_system.iterrows():
                        if row['start'] < timestamp <= row['end']:
                            label_assigned = row['attack']
                            break

                    # Cập nhật nhãn vào JSON
                    data_json["Label"] = label_assigned

                    # Ghi dòng JSON đã cập nhật vào file output
                    with open(os.path.join(output_path, f"{file}_labeled.json"), 'a') as f_out:
                        f_out.write(json.dumps(data_json) + "\n")



In [18]:

resp = client.indices.put_index_template(
    name="aminer-template",
    index_patterns=[
        "aminer-*"
    ],
    template={
        "settings": {
            "number_of_shards": 1,
            "number_of_replicas": 1
        },
        "mappings": {
            "properties": {
                "AnalysisComponent": {
                    "properties": {
                        "AnalysisComponentIdentifier": {"type": "integer"},
                        "AnalysisComponentType": {"type": "keyword"},
                        "AnalysisComponentName": {"type": "text"},
                        "Message": {"type": "text"},
                        "PersistenceFileName": {"type": "keyword"},
                        "TrainingMode": {"type": "boolean"},
                        "AffectedLogAtomPaths": {"type": "keyword"},
                    }
                },
                "LogData": {
                    "properties": {
                        "RawLogData": {"type": "text"},
                        "Timestamps": {"type": "date", "format": "epoch_second"},
                        "DetectionTimestamp": {"type": "date", "format": "epoch_second"},
                        "LogLinesCount": {"type": "integer"},
                        "LogResources": {"type": "keyword"}
                    }
                },
                "AMiner": {
                    "properties": {
                        "ID": {"type": "ip"}  # Nếu ID không phải là IP, đổi sang keyword
                    }
                },
                "Label": {"type": "keyword"}  # Nhãn của dữ liệu
            }
        }
    },
    priority=102,
)
print(resp)

{'acknowledged': True}


In [23]:

resp = client.indices.put_index_template(
    name="wazuh-template",
    index_patterns=["wazuh-*"],  # Index pattern phù hợp
    template={
        "settings": {
            "number_of_shards": 1,
            "number_of_replicas": 1
        },
        "mappings": {
            "properties": {
                "@timestamp": {"type": "date"},  # Trường thời gian bắt buộc
                "predecoder": {
                    "properties": {
                        "hostname": {"type": "keyword"},
                        "program_name": {"type": "keyword"},
                        "timestamp": {"type": "date"}  # Có thể chuyển thành "date" nếu cần
                    }
                },
                "agent": {
                    "properties": {
                        "ip": {"type": "ip"},  # Kiểu IP
                        "name": {"type": "keyword"},
                        "id": {"type": "keyword"}
                    }
                },
                "manager": {
                    "properties": {
                        "name": {"type": "keyword"}
                    }
                },
                "rule": {
                    "properties": {
                        "firedtimes": {"type": "integer"},
                        "mail": {"type": "boolean"},
                        "level": {"type": "integer"},
                        "pci_dss": {"type": "keyword"},  # Mảng các giá trị keyword
                        "tsc": {"type": "keyword"},
                        "description": {"type": "text"},
                        "groups": {"type": "keyword"},
                        "id": {"type": "keyword"},
                        "nist_800_53": {"type": "keyword"},
                        "gpg13": {"type": "keyword"},
                        "gdpr": {"type": "keyword"}
                    }
                },
                "decoder": {
                    "properties": {
                        "name": {"type": "keyword"}
                    }
                },
                "full_log": {"type": "text"},  # Trường văn bản
                "input": {
                    "properties": {
                        "type": {"type": "keyword"}
                    }
                },
                "location": {"type": "keyword"},  # Đường dẫn file log
                "id": {"type": "keyword"}  # ID của log
            }
        }
    },
    priority=102  # Ưu tiên của template
)

print(resp)

{'acknowledged': True}


In [7]:
def import_json_to_es(file_path, index_name):
     # with open(file_path, "r", encoding="utf-8") as f:
     #    # count = 0
     #    for line in f:
     #        # count += 1
     #        # if count == 100:
     #        #     break
     #        doc = json.loads(line.strip())
     #        client.index(index=index_name, body=doc)
     actions = []
     chunk_size = 1000  # Kích thước chunk để gửi bulk request

     with open(file_path, "r", encoding="utf-8") as f:
        for line_number, line in enumerate(f, 1):
            try:
                doc = json.loads(line.strip())
                action = {
                    "_index": index_name,
                    "_source": doc
                }
                actions.append(action)

                # Send bulk request when chunk size is reached
                if len(actions) >= chunk_size:
                    helpers.bulk(client, actions)
                    actions = []

            except json.JSONDecodeError as e:
                print(f"Error decoding JSON on line {line_number}: {e}")
                continue

        # Send any remaining documents
        if actions:
            helpers.bulk(client, actions)


In [21]:
import_json_to_es("data/AIT_labeled/harrison_aminer_labeled.json", "aminer-harrison")
import_json_to_es("data/AIT_labeled/russellmitchell_aminer_labeled.json", "aminer-russellmitchell")
import_json_to_es("data/AIT_labeled/santos_aminer_labeled.json", "aminer-santos")
import_json_to_es("data/AIT_labeled/shaw_aminer_labeled.json", "aminer-shaw")
import_json_to_es("data/AIT_labeled/wardbeck_aminer_labeled.json", "aminer-wardbeck")
import_json_to_es("data/AIT_labeled/wheeler_aminer_labeled.json", "aminer-wheeler")
import_json_to_es("data/AIT_labeled/wilson_aminer_labeled.json", "aminer-wilson")


FileNotFoundError: [Errno 2] No such file or directory: 'data/AIT_labeled/wilson_aminer_labeled.json.json'

In [8]:
import_json_to_es("data/AIT_labeled/fox_wazuh_labeled.json", "wazuh-fox")
import_json_to_es("data/AIT_labeled/harrison_wazuh_labeled.json", "wazuh-harrison")
import_json_to_es("data/AIT_labeled/russellmitchell_wazuh_labeled.json", "wazuh-russellmitchell")
import_json_to_es("data/AIT_labeled/santos_wazuh_labeled.json", "wazuh-santos")
import_json_to_es("data/AIT_labeled/shaw_wazuh_labeled.json", "wazuh-shaw")
import_json_to_es("data/AIT_labeled/wardbeck_wazuh_labeled.json", "wazuh-wardbeck")
import_json_to_es("data/AIT_labeled/wheeler_wazuh_labeled.json", "wazuh-wheeler")
import_json_to_es("data/AIT_labeled/wilson_wazuh_labeled.json", "wazuh-wilson")



In [10]:
from sqlmodel import create_engine

DATABASE_URL = f"postgresql://{os.getenv("DB_USER")}:{os.getenv("DB_PASSWORD")}@{os.getenv("DB_HOST")}:{os.getenv("DB_PORT")}/{os.getenv("DB_NAME")}"

engine = create_engine(DATABASE_URL, echo=True)

if not engine:
    raise ConnectionError("Database engine not initialized")

In [20]:
folder_extracted ="data/AIT_extracted"
file_extracted = get_all_file_names(folder_extracted)
print(file_extracted)

['fox_aminer', 'harrison_aminer', 'russellmitchell_aminer', 'santos_aminer', 'shaw_aminer', 'wardbeck_aminer', 'wheeler_aminer', 'wilson_aminer']


In [21]:
def swap_string(s):
    if '_' in s:
        parts = s.split('_')
        if len(parts) == 2:
            return f"{parts[1]}_{parts[0]}"
    return s  # Trả lại chuỗi gốc nếu không đúng định dạng


In [22]:
for file in  file_extracted:
    with open(f"{folder_extracted}/{file}.csv", "r", encoding="utf-8") as f:
        data = pd.read_csv(f)
        data.to_sql(swap_string(file), engine, if_exists="replace", index=False)


2025-04-26 10:10:14,776 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2025-04-26 10:10:14,777 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-04-26 10:10:14,786 INFO sqlalchemy.engine.Engine select current_schema()
2025-04-26 10:10:14,787 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-04-26 10:10:14,794 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2025-04-26 10:10:14,794 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-04-26 10:10:14,797 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-04-26 10:10:14,886 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname